# Experiment README

## Table of Contents

* [Overview of Experiment Architecture](#Overview-of-Experiment-Architecture)
* [Experiment Workflow](#Experiment-Workflow)
    * [Modifying State Variables](#Modifying-State-Variables)
    * [Modifying System Parameters](#Modifying-System-Parameters)
    * [Executing Experiments](#Executing-Experiments)
    * [Post-processing and Analysing Results](#Post-processing-and-Analysing-Results)
    * [Visualizing Results](#Visualizing-Results)
* [Creating New, Customized Experiment Notebooks](#Creating-New,-Customized-Experiment-Notebooks)
    * Step 1: Select an experiment template
    * Step 2: Create a new notebook
    * Step 3: Customize the experiment
    * Step 4: Execute the experiment
* [Advanced Experiment-configuration & Simulation Techniques](#Advanced-Experiment-configuration-&-Simulation-Techniques)
    * [Setting Simulation Timesteps and Unit of Time `dt`](#Setting-Simulation-Timesteps-and-Unit-of-Time-dt)
    * [Changing the Ethereum Network Upgrade Stage](#Changing-the-Ethereum-Network-Upgrade-Stage)
    * [Performing Large-scale Experiments](#Performing-Large-scale-Experiments)

# Overview of Experiment Architecture

The experiment architecture is composed of the following four elements – the **model**, **default experiment**, **experiment templates**, and **experiment notebooks**:

1. The **model** is initialized with a default Initial State and set of System Parameters defined in the `model` module.
2. The **default experiment** – in the `experiments.default_experiment` module – is an experiment composed of a single simulation that uses the default cadCAD **model** Initial State and System Parameters. Additional default simulation execution settings such as the number of timesteps and runs are also set in the **default experiment**.
3. The **experiment templates** – in the `experiments.templates` module – contain pre-configured analyses based on the **default experiment**. Examples include... To be created!
4. The **experiment notebooks** perform various scenario analyses by importing existing **experiment templates**, optionally modifying the Initial State and System Parameters within the notebook, and then executing them.

# Experiment Workflow

If you just want to run (execute) existing experiment notebooks, simply open the respective notebook and execute all cells.

Depending on the chosen template and planned analysis, the required imports might differ slightly from the below standard dependencies:

In [ ]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook-configuration methods, such as loading IPython modules
import setup

# External dependencies
from pprint import pprint
import copy
import logging
import pandas as pd
import importlib as imp
import matplotlib
import matplotlib.pyplot as plt

# Project dependencies
import model.constants as constants
from experiments.run import run
import experiments.visualizations as visualizations
import experiments.default_experiment as default_experiment

# types
from model.types import *
from model.entities.balance import Balance

# options
pd.options.plotting.backend = 'matplotlib'
%matplotlib inline

We can then create a copy of the simulation object – we create a new copy for each analysis we'd like to perform:

In [ ]:
simulation_analysis_1 = copy.deepcopy(default_experiment.experiment.simulations[0])

## Modifying State Variables

To view what the Initial State (radCAD model-configuration setting `initial_state`) of the State Variables are, and to what value they have been set, we can inspect the dictionary as follows:

In [ ]:
pprint(simulation_analysis_1.model.initial_state)
# simulation_analysis_1.model.state

To modify the value of **State Variables** for a specific analysis, you need to select the relevant simulation and update the chosen model Initial State. For example, updating the `floating_supply` Initial State to `100e6` CELO and `123e5` cUSD.

In [ ]:
simulation_analysis_1.model.initial_state.update({
    'reserve_balance': Balance({
        CryptoAsset.CELO: 120.0e6,
        Stable.CUSD: 0.0
    }),
})

In [ ]:
pprint(simulation_analysis_1.model.initial_state)

## Modifying System Parameters

To view what the System Parameters (radCAD model configuration setting `params`) are, and to what value they have been set, we can inspect the dictionary as follows:

In [ ]:
pprint(simulation_analysis_1.model.params)

To modify the value of **System Parameters** for a specific analysis, you need to select the relevant simulation, and update the chosen model System Parameter (which is a list of values). For example, updating the `reserve_fraction` System Parameter to a sweep of two values, `0.001` and `0.01`:

In [ ]:
simulation_analysis_1.model.params.update({
    "reserve_fraction": [0.5],
})

## Executing Experiments

We can now execute our custom analysis and retrieve the post-processed Pandas DataFrame using the `run(...)` method:

In [ ]:
df, exceptions = run(simulation_analysis_1)

## Post-processing and Analysing Results

We can see that we had no exceptions for the single simulation we executed:

In [ ]:
exceptions[0]['exception'] == None

We can simply display the Pandas DataFrame to inspect the results. This DataFrame already has some default post-processing applied (see [experiments/post_processing.py](../post_processing.py)). For example, parameters that change in the parameter grid (if there are any) are attached as columns to the end of the dataframe.

In [ ]:
df.head()

In [ ]:
pprint(df['subset'].unique())
pprint(df['run'].unique())

In [ ]:
# distinguish substeps over time
# requires `experiment.engine.drop_substeps = True`in default_experiment.py
#df['scaled_timestep'] = df['timestep']*10+df['substep']

## Visualizing Results

Once we have the results post-processed and in a Pandas DataFrame, we can use Plotly for plotting our results (here two subsets because of the `reserve_fraction` parameter sweep introduced above):

- Celo market price
- cUSD market price
- Oracle rate
- Floating supply Celo
- floating supply cusd
- Mento buckets celo cusd

In [ ]:
# timestep window
#df = df[df.index<250]

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='market_price_celo_usd')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='market_price_cusd_usd')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='oracle_rate_celo_usd')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='floating_supply_celo', y_label='# tokens')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='floating_supply_cusd', y_label='# tokens')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='mento_buckets_cusd_celo.reserve_asset', y_label='# tokens')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='mento_buckets_cusd_celo.stable', y_label='# tokens')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='reserve_balance_celo', y_label='# tokens')

In [ ]:
visualizations.plot_helper(data_frame=df, column_label='mento_rate_cusd_celo')